# IRONFORGE Temporal Query Explorer
Interactive notebook for querying temporal patterns in NetworkX graphs

In [ ]:
# Import the query engine
from temporal_query_engine import TemporalQueryEngine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Initialize the engine
engine = TemporalQueryEngine()
print(f"✅ Loaded {len(engine.sessions)} sessions for analysis")

## Quick Start: Ask Your Questions

In [ ]:
# Ask temporal questions
result = engine.ask("What happens after a 40% zone event?")
result

In [ ]:
# Analyze opening patterns
result = engine.ask("When session starts with expansion, what's the final range?")

# Visualize the results
if 'pattern_outcomes' in result:
    patterns = result['pattern_outcomes']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Pattern frequency
    pattern_names = list(patterns.keys())
    pattern_counts = [patterns[p]['count'] for p in pattern_names]
    
    ax1.bar(pattern_names, pattern_counts)
    ax1.set_title('Opening Pattern Frequency')
    ax1.set_ylabel('Number of Sessions')
    plt.setp(ax1.get_xticklabels(), rotation=45)
    
    # Average ranges by pattern
    avg_ranges = [patterns[p]['avg_range'] for p in pattern_names]
    
    ax2.bar(pattern_names, avg_ranges, color='orange')
    ax2.set_title('Average Final Range by Opening Pattern')
    ax2.set_ylabel('Points')
    plt.setp(ax2.get_xticklabels(), rotation=45)
    
    plt.tight_layout()
    plt.show()

## Advanced Graph Analysis

In [ ]:
# Analyze a specific session's graph structure
session_id = list(engine.sessions.keys())[0]  # Pick first session
graph = engine.graphs[session_id]
nodes = engine.sessions[session_id]

print(f"📊 Analyzing session: {session_id}")
print(f"Nodes: {graph.number_of_nodes()}, Edges: {graph.number_of_edges()}")

# Graph metrics
if graph.number_of_nodes() > 0:
    density = nx.density(graph)
    print(f"Graph density: {density:.3f}")
    
    # Degree distribution
    degrees = dict(graph.degree())
    plt.figure(figsize=(10, 6))
    plt.hist(list(degrees.values()), bins=20, alpha=0.7)
    plt.title(f'Degree Distribution - {session_id}')
    plt.xlabel('Node Degree')
    plt.ylabel('Frequency')
    plt.show()

## Custom Temporal Queries

In [ ]:
# Define your own custom query function
def analyze_price_momentum(session_id, window_minutes=10):
    """Analyze price momentum patterns in a session"""
    nodes = engine.sessions[session_id]
    graph = engine.graphs[session_id]
    
    # Calculate momentum windows
    window_ms = window_minutes * 60 * 1000
    
    momentum_events = []
    for i in range(0, len(nodes) - 1):
        current_node = nodes.iloc[i]
        future_nodes = nodes[
            (nodes['t'] > current_node['t']) & 
            (nodes['t'] <= current_node['t'] + window_ms)
        ]
        
        if len(future_nodes) > 0:
            price_change = future_nodes['price'].iloc[-1] - current_node['price']
            momentum_events.append({
                'time': current_node['t'],
                'price': current_node['price'],
                'momentum': price_change,
                'events_ahead': len(future_nodes)
            })
    
    return pd.DataFrame(momentum_events)

# Analyze momentum for a session
momentum_df = analyze_price_momentum(session_id)
print(f"🎯 Momentum analysis for {session_id}:")
print(momentum_df.describe())

# Plot momentum over time
plt.figure(figsize=(12, 6))
plt.scatter(range(len(momentum_df)), momentum_df['momentum'], alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.title(f'Price Momentum Over Time - {session_id}')
plt.xlabel('Event Sequence')
plt.ylabel('Price Change (Points)')
plt.show()

## Interactive Query Widget

In [ ]:
# Create interactive query widget
from ipywidgets import interact, widgets

@interact
def interactive_query(
    question=widgets.Text(
        value="What happens after a 40% zone event?",
        description="Question:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
):
    if question.strip():
        result = engine.ask(question)
        
        print(f"🤔 Query: {question}")
        print(f"🎯 Results:")
        
        for key, value in result.items():
            if isinstance(value, list) and len(value) > 3:
                print(f"  {key}: {len(value)} items (showing first 3)")
                for item in value[:3]:
                    print(f"    • {item}")
            else:
                print(f"  {key}: {value}")

## Session Comparison Tool

In [ ]:
# Compare multiple sessions
def compare_sessions(session_ids):
    """Compare temporal patterns across multiple sessions"""
    comparison = []
    
    for session_id in session_ids:
        if session_id in engine.sessions:
            info = engine.session_info(session_id)
            comparison.append(info)
    
    return pd.DataFrame(comparison)

# Compare first 5 sessions
sample_sessions = list(engine.sessions.keys())[:5]
comparison_df = compare_sessions(sample_sessions)

print("📊 Session Comparison:")
display(comparison_df[['session_id', 'nodes', 'edges', 'price_range']])

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Price ranges
axes[0,0].bar(range(len(comparison_df)), comparison_df['price_range'])
axes[0,0].set_title('Price Ranges by Session')
axes[0,0].set_ylabel('Points')

# Node counts
axes[0,1].bar(range(len(comparison_df)), comparison_df['nodes'])
axes[0,1].set_title('Event Counts by Session')
axes[0,1].set_ylabel('Number of Events')

# Edge counts
axes[1,0].bar(range(len(comparison_df)), comparison_df['edges'])
axes[1,0].set_title('Edge Counts by Session')
axes[1,0].set_ylabel('Number of Edges')

# Range vs Events scatter
axes[1,1].scatter(comparison_df['nodes'], comparison_df['price_range'])
axes[1,1].set_title('Price Range vs Event Count')
axes[1,1].set_xlabel('Number of Events')
axes[1,1].set_ylabel('Price Range (Points)')

plt.tight_layout()
plt.show()